In [1]:
catchment_name = 'LinDyke' #LinDyke
methods_key ='Observed'

# Something in setting crop to to true for Wyke Beck doesn't work (the land cover and results files have different
# values for out_meta so end up different sizes)
if catchment_name =='WykeBeck':
    crop_or_not = False
elif catchment_name == 'LinDyke':
    crop_or_not = True    

region = '' # 'Kippax' #'' # 'Garforth'

In [2]:
# Set up link to model directory and read in catchment shapefile
model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles_export/'.format(catchment_name, methods_key)
landcover_directory = '../../../FloodModelling/{}Models/LandCoverData/'.format(catchment_name)

# Define whether to filter out values <0.1
remove_little_values = True

In [3]:
import pandas as pd
import numpy as np
import sys
import geopandas as gpd

sys.path.append("../")
from my_functions import *

# Specify strings relating to catchment
if catchment_name == 'LinDyke':
    catchment_name_str = "Resampled.Terrain" 
    # catchment_gdf = gpd.read_file(model_directory + 'CatchmentLinDyke_exported.shp')
    catchment_gdf = gpd.read_file(os.path.join(model_directory, '../SubCatchmentBoundaries/ManualTrimAboveWetlands.shp'))
    cell_size_in_m2 = 1
elif catchment_name == 'WykeBeck':
    catchment_name_str = "Terrain.wykeDEM" 
    cell_size_in_m2 = 4
    catchment_gdf = gpd.read_file(model_directory + 'WykeBeckCatchment.shp')

### Define the names of the method (in dictionary for different model runs)

In [4]:
methods_dict = {'Idealised': [ '6h_sp_c_0.5','6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4',
                    '6h_sp_bl_0.6','6h_sp_bl_0.7','6h_sp_bl_0.8','6h_sp_bl_0.9'],
                'Observed':['6h_feh_singlepeak', '6h_c1','6h_c2','6h_c3','6h_c4', '6h_c5', '6h_c6','6h_c7',
             '6h_c8','6h_c9','6h_c10', '6h_c11', '6h_c12','6h_c13','6h_c14','6h_c15'], 
               'SinglePeak_Scaled':['6h_sp_+0%','6h_sp_+5%','6h_sp_+10%','6h_sp_+15%','6h_sp_+20%']}

In [5]:
methods = methods_dict[methods_key]

### Get version of landcover array with just 'urban' and 'rural' categories

In [6]:
# Water landcover classification - 10 is water, 11 is eveyrthing else
with rasterio.open(landcover_directory + 'LandCover_notwater_classification.tif', 'r') as ds:
    landcover_notwater = ds.read()[0]
    out_meta = ds.meta
landcover_notwater_flat = landcover_notwater.flatten()

# Urban landcover classification - 10 is urban, 1 is everything else
with rasterio.open(landcover_directory + 'LandCover_urban_and_suburban_classification.tif', 'r') as ds:
    landcover_urban = ds.read()[0]
landcover_urban_flat = landcover_urban.flatten()

### Find maximum intensity for each method and minute in which it occurs (to use in sorting results analysis)

In [7]:
maxs = []
min_of_maxs = []

for method in methods:
    if method == '6h_feh_singlepeak':
        precip=pd.read_csv("../CreateSyntheticRainfallEvents/FEHProfiles/{}/6hr_100yrRP/PostLossRemoval/6hr_100yrRP_6.01h_1mintimestep.csv".format(catchment_name))
    elif method  == '6h_c12_copy':
        method = '6h_c12'
        precip=pd.read_csv("../CreateSyntheticRainfallEvents/{}Profiles/{}/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(methods_key,catchment_name, method))
    else:
        precip=pd.read_csv("../CreateSyntheticRainfallEvents/{}Profiles/{}/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(methods_key,catchment_name, method))
    # Trim and add minutes column
    precip = precip[0:360].copy()
    precip['minute']=range(1,361)
    # Add max and minutes of max
    maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
    min_of_maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

### Create versions of lists of methods, in order based on max intensity and the the timing of the max intensity 

In [8]:
short_ids_by_loading=  pd.DataFrame({"min": min_of_maxs, 'method_name': methods}).sort_values('min')["method_name"].tolist()
short_ids_by_intensity = pd.DataFrame({"min": maxs, 'method_name': methods}).sort_values('min', ascending = False)["method_name"].tolist()

### Create dataframe of colours for each cluster (based on their loading)

In [9]:
if methods_key == 'Observed':
    colours_df = create_colours_df_observed(short_ids_by_loading, methods)
elif methods_key == 'Idealised':
    colours_df = create_colours_df_idealised( short_ids_by_loading, methods)
elif methods_key == 'SinglePeak_Scaled':
    colours_df = create_colours_df_sp( short_ids_by_loading, methods)

### Create list of filepaths, formatted to be used for either depth or velocity

In [10]:
fps = []
for method_num, short_id in enumerate(methods):
    fp = model_directory + "{}/{} (Max).{}.tif".format(short_id, '{}', catchment_name_str)
    fps.append(fp)
if methods_key == 'Observed':
    fps[0] = '../../../FloodModelling/{}Models/Model_FEHProfiles_export/6h_feh_singlepeak/{}/{} (Max).{}.tif'.format(catchment_name, region, '{}', catchment_name_str)

# Hazard calculations
The hazard is calculated based on definition here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/842485/What-is-the-Risk-of-Flooding-from-Surface-Water-Map.pdf . 

### Calculate the hazard categories for the single peak results

In [11]:
# Read in depth and velocity rasters
sp_depth, out_meta = open_and_clip_to_catchment(fps[0].format('Depth'), catchment_gdf, crop_or_not)
sp_velocity, out_meta = open_and_clip_to_catchment(fps[0].format('Velocity'),  catchment_gdf, crop_or_not)

if remove_little_values == True:
    sp_depth = remove_little_values_fxn(sp_depth, fp.format('Depth'), catchment_gdf, crop_or_not) 
    sp_velocity = remove_little_values_fxn(sp_velocity, fp.format('Velocity'), catchment_gdf, crop_or_not)    
    
# Create a composite hazard array
sp_hazard= np.where((sp_depth.flatten())<0.25, (sp_depth.flatten()* (sp_velocity.flatten()+0.5) + 0.5) , (sp_depth.flatten() * (sp_velocity.flatten()+0.5) + 1))
sp_hazard = sp_hazard.reshape(sp_depth.shape)    

# Classify this according to bins
breaks_hazard = np.array([0, 0.5, 0.75, 1.25,2, 100])  

sp_classified_hazard = classify_raster(sp_hazard, breaks_hazard)
# Save to file
fp_for_hazard = fps[0].replace('{} (Max).{}'.format({}, catchment_name_str),'hazard_classified')
save_array_as_raster(sp_classified_hazard, fp_for_hazard, out_meta) 

# Plot the classified hazard categories
labels_hazard = ['Low hazard', 'Moderate hazard', 'Significant hazard', 'Extreme hazard']
classified_colors_list_hazard = [mpl.cm.autumn_r(0.2),mpl.cm.autumn_r(0.5), mpl.cm.autumn_r(0.7),"darkred"]
plot_classified_raster(fp_for_hazard, labels_hazard, classified_colors_list_hazard, catchment_gdf, catchment_name ,methods_key) 

### Calculate the hazard categories for the other scenarios, and then find the difference in the categories between those and single peak

In [12]:
for fp in fps[1:]:
    print(fp)

    # Read in depth and velocity rasters
    depth, out_meta = open_and_clip_to_catchment(fp.format('Depth'), catchment_gdf, crop_or_not)
    velocity, out_meta = open_and_clip_to_catchment(fp.format('Velocity'),  catchment_gdf, crop_or_not)

    if remove_little_values == True:
        depth = remove_little_values_fxn(depth, fp.format('Depth'), catchment_gdf, crop_or_not) 
        velocity = remove_little_values_fxn(velocity, fp.format('Velocity'), catchment_gdf, crop_or_not)    

    # Create a composite hazard array
    hazard= np.where((depth.flatten())<0.25, (depth.flatten()* (velocity.flatten()+0.5) + 0.5) , (depth.flatten() * (velocity.flatten()+0.5) + 1))
    hazard = hazard.reshape(depth.shape)    
    
    # Classify this according to bins
    classified_hazard = classify_raster(hazard, breaks_hazard)
    # Save to file
    fp_for_hazard = fp.replace('{} (Max).{}'.format({}, catchment_name_str),'{}_classified'.format('hazard') )
    save_array_as_raster(classified_hazard, fp_for_hazard, out_meta) 
    # Plot the classified hazard categories
    labels_hazard = ['Low hazard', 'Moderate hazard', 'Significant hazard', 'Extreme hazard']
    classified_colors_list_hazard = [mpl.cm.autumn_r(0.2),mpl.cm.autumn_r(0.5), mpl.cm.autumn_r(0.7)]
    plot_classified_raster(fp_for_hazard, labels_hazard, classified_colors_list_hazard, catchment_gdf, catchment_name ,methods_key)   

../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c1/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c2/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c3/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c4/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c5/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c6/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c7/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c8/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedProfiles_export/6h_c9/{} (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_ObservedPro

# <u> Flood extent </u>
To examine whether the rainfall's temporal distribution influences the total extent of flooding, the number of flooded cells and the total flooded area in km2 (incl. only cells with depth >0.1m) is compared between the profile with a single peak, and the three methods for producing multi-peaked rainfall events. b

### Create dataframes containing the (total/urban) flooded area in each depth/velocity bin

In [15]:
# Define breaks to split the depths/velocities on
breaks_depths = np.array([0, 0.3, 0.6, 1.2, 100])  
labels_depth = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
breaks_velocity = np.array([0,0.25,0.5,2,100])
labels_velocity = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]

In [16]:
velocity_counts, velocity_props = create_binned_counts_and_props(methods, fps, '', 'Velocity',catchment_gdf, crop_or_not=crop_or_not,
                                                                remove_little_values = remove_little_values)
depth_counts, depth_props  = create_binned_counts_and_props(methods, fps, '', 'Depth',catchment_gdf, crop_or_not = crop_or_not,
                                                           remove_little_values = remove_little_values)

In [17]:
velocity_counts_urban, velocity_props_urban = create_binned_counts_and_props(methods, fps, True,'Velocity', catchment_gdf,
                                                         crop_or_not, landcover_urban_flat,  remove_little_values = remove_little_values)
depth_counts_urban, depth_props_urban= create_binned_counts_and_props(methods, fps, True,'Depth',  catchment_gdf, 
                                                      crop_or_not, landcover_urban_flat, remove_little_values = remove_little_values)

In [18]:
velocity_counts_notwater, velocity_props_notwater = create_binned_counts_and_props(methods, fps, True,'Velocity', catchment_gdf,
                                      crop_or_not, landcover_notwater_flat, remove_little_values = remove_little_values)
depth_counts_notwater, depth_props_notwater = create_binned_counts_and_props(methods, fps, True,'Depth',catchment_gdf, 
                                    crop_or_not, landcover_notwater_flat, remove_little_values = remove_little_values)

### Create dataframes containing the (total/urban) flooded area

In [19]:
totals_df = create_totals_df(velocity_counts, cell_size_in_m2)

In [20]:
totals_df_urban = create_totals_df(velocity_counts_urban, cell_size_in_m2)  
totals_df_notwater = create_totals_df(velocity_counts_notwater, cell_size_in_m2)  

### Create dataframes containing the % diff in the flooded area between single peak and each other method  

In [21]:
if methods_key == 'Observed':
    column_for_comparison = '6h_feh_singlepeak'
elif methods_key == 'Idealised':
    column_for_comparison ='6h_sp_c_0.5'    
elif methods_key == 'SinglePeak_Scaled':
    column_for_comparison ='6h_sp_+0%'        
    
percent_diffs_df = find_percentage_diff (methods, column_for_comparison, totals_df, fps) 
percent_diffs_df_urban = find_percentage_diff (methods, column_for_comparison, totals_df_urban, fps)
percent_diffs_df_notwater = find_percentage_diff (methods, column_for_comparison, totals_df_notwater, fps)

## Find number of cells with each hazard rating

In [22]:
hazard_counts, hazard_props = create_binned_counts_and_props_hazard(methods, fps, '', catchment_name_str,catchment_gdf, 
                                                                    crop_or_not, landcover_data=False, remove_little_values =remove_little_values )
hazard_counts_urban, hazard_props_urban = create_binned_counts_and_props_hazard(methods, fps, 'Urban', catchment_name_str,
                                                                catchment_gdf, crop_or_not, landcover_urban, remove_little_values = remove_little_values)
hazard_counts_notwater, hazard_props_notwater = create_binned_counts_and_props_hazard(methods, fps, 'Notwater', 
                                                    catchment_name_str,catchment_gdf, crop_or_not, landcover_notwater, remove_little_values = remove_little_values)

In [40]:
hazard_counts_notwater

,index,6h_feh_singlepeak,6h_c1,6h_c2,6h_c3,6h_c4,6h_c5,6h_c6,6h_c7,6h_c8,6h_c9,6h_c10,6h_c11,6h_c12,6h_c13,6h_c14,6h_c15
0,Low hazard,231129,273119,200362,232503,193337,293670,201872,232873,153019,186838,208267,158083,182634,193239,238517,227869
1,Moderate hazard,67885,75205,56569,65118,54227,74328,58457,62562,48538,54397,59935,48535,52559,55904,65623,65866
2,Significant hazard,181454,202278,157624,177030,149838,215004,160992,175148,132380,150107,162535,135539,146114,153834,183620,174737
3,Extreme hazard,29381,31669,26435,28351,24463,35066,25959,30621,17059,22830,25245,19575,23892,23077,31873,25270


In [41]:
hazard_counts

,index,6h_feh_singlepeak,6h_c1,6h_c2,6h_c3,6h_c4,6h_c5,6h_c6,6h_c7,6h_c8,6h_c9,6h_c10,6h_c11,6h_c12,6h_c13,6h_c14,6h_c15
0,Low hazard,235335,277475,204083,236572,196884,297040,205693,236574,155604,190445,212186,160793,185830,196945,242199,232072
1,Moderate hazard,70962,79170,58591,68405,55906,78963,60695,65978,49380,56053,62467,49530,53975,57678,69380,68957
2,Significant hazard,183752,205547,159635,179378,151738,219556,162898,177727,133550,151839,164404,136937,147898,155618,186334,176861
3,Extreme hazard,29407,31695,26461,28376,24488,35102,25983,30648,17071,22848,25265,19590,23916,23101,31904,25293


### Create a dataframe containing all the info on each of the scenarios

In [28]:
cluster_results = pd.DataFrame({'Cluster_num': methods, "MaxRainfallIntensity": maxs,  
    "MaxRainfallIntensityMinute": min_of_maxs,
    # All cells
   'FloodedArea':totals_df['FloodedArea'],
    '%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs'],
    '%Diff_FloodedArea_fromSP_formatted':percent_diffs_df['percent_diff_formatted'],
    'Abs%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs_abs'],
    # Urban cells
 'UrbanFloodedArea':totals_df_urban['FloodedArea'],
 '%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs'] ,
  '%Diff_UrbanFloodedArea_fromSP_formatted':percent_diffs_df_urban['percent_diff_formatted'],
   'Abs%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs_abs'], 
    # Not water cells
 'NotwaterFloodedArea':totals_df_notwater['FloodedArea'],
 '%Diff_NotwaterFloodedArea_fromSP':percent_diffs_df_notwater['percent_diffs'] ,
  '%Diff_NotwaterFloodedArea_fromSP_formatted':percent_diffs_df_notwater['percent_diff_formatted'],
   'Abs%Diff_NotwaterFloodedArea_fromSP':percent_diffs_df_notwater['percent_diffs_abs'],                                          
   #'WorstCaseDepth_ncells': worst_case_method_depth['counts'].tolist(),
   # 'WorstCaseVelocity_ncells': worst_case_method_velocity['counts'].tolist(), 
                                'colour':colours_df['colour']}) 

### Add the depth/velocity category breakdowns and hazard categories to this

In [29]:
dfs = [velocity_props, depth_props,  velocity_props_urban, depth_props_urban,  velocity_props_notwater, depth_props_notwater,   
       velocity_counts, depth_counts, velocity_counts_urban, depth_counts_urban, velocity_counts_notwater, depth_counts_notwater,  
       hazard_counts, hazard_props, hazard_counts_urban, hazard_props_urban, hazard_counts_notwater, hazard_props_notwater, ]
suffixes = ['_propcells', '_propcells', '_propcells_urban','_propcells_urban','_propcells_notwater','_propcells_notwater',
            '_countcells','_countcells','_countcells_urban', '_countcells_urban','_countcells_notwater', '_countcells_notwater', '_countcells_notwater', '_countcells_notwater',
            '_countcells', '_propcells',  '_countcells_urban', '_propcells_urban', '_countcells_notwater', '_propcells_notwater', ]

for num, df in enumerate(dfs):
    # Reformat the dataframe
    df = df.set_index('index').T
    # Add the correct suffix to the column names
    df = df.add_suffix(suffixes[num]) 
    # Add Cluster_num column for joining
    df['Cluster_num'] = df.index#
    # Join to cluster results dataframe
    cluster_results = pd.merge(cluster_results,  df, how="outer", on = 'Cluster_num')
    
# cluster_results = pd.merge(cluster_results, hazard_cat_changes,  how="outer", on = 'Cluster_num')    

### Finding proportion of area/urban area flooded

In [28]:
# cluster_results['%floodedarea_urban'] = round(cluster_results['UrbanFloodedArea']/cluster_results['FloodedArea']*100,2)
# cluster_results['%_of_area_flooded'] =(cluster_results['FloodedArea']/29.589)*100
# cluster_results['%_of_urban_area_flooded'] =(cluster_results['UrbanFloodedArea']/7.987)*100
# # Add NAs for SP
# cluster_results['%Diff_FloodedArea_fromSP_formatted']=cluster_results['%Diff_FloodedArea_fromSP_formatted'].fillna('')
# cluster_results['%Diff_UrbanFloodedArea_fromSP_formatted']=cluster_results['%Diff_UrbanFloodedArea_fromSP_formatted'].fillna('')

## Summarise the number of cells in different depth/velocity categories

#### Get one dataframe containing the values for all methods, one row per cell per method 
Also including the water class variable in that cell

In [25]:
each_cells_value = produce_df_of_cell_by_cell_values(model_directory, catchment_gdf, catchment_name_str, methods, 
                                                     landcover_notwater_flat,landcover_urban_flat, crop_or_not,
                                                    remove_little_values = remove_little_values)
# rename for consistency
each_cells_value['short_id'] = each_cells_value['short_id'].map({'6h_feh_singlepeak': 'FEH'}).fillna(each_cells_value['short_id'] )

In [46]:
np.unique(each_cells_value[each_cells_value['short_id']=='6h_c5']['Hazard'], return_counts=True)

(array([ 2.,  3.,  4.,  5., nan]),
 array([297040,  78963, 219556,  35102, 490260]))

### Rename the profile names

In [30]:
if methods_key == 'Idealised':
    cluster_results['Cluster_num']=['C', 'FL1', 'FL2', 'FL3', 'FL4','BL6', 'BL7', 'BL8','BL9']
    ### Reorder to C in middle
    cluster_results = cluster_results.reindex([1,2,3,4,0,5,6,7,8])
    cluster_results.reset_index(inplace=True, drop=True)
if methods_key == 'Observed':
    methods = ['6h_feh_singlepeak','6h_c1','6h_c8','6h_c15','6h_c3','6h_c11','6h_c10','6h_c9','6h_c13','6h_c6',
                 '6h_c2','6h_c12','6h_c14','6h_c4','6h_c7','6h_c5']
    cluster_results = cluster_results.reindex(cluster_results['Cluster_num'].map(dict(zip(methods, range(len(methods))))).sort_values().index)
    cluster_results.reset_index(inplace=True, drop=True)
    cluster_results['Cluster_num'] = cluster_results['Cluster_num'].map({'6h_feh_singlepeak': 'FEH'}).fillna(cluster_results['Cluster_num'] )

### Save to file

In [31]:
# Create path to the folder
path = "Outputs/Data/{}Profiles/{}/".format(methods_key, catchment_name)
# Check whether the specified path exists or not
isExist = os.path.exists(path)
# Create a new directory because it does not exist
if not isExist:
    os.makedirs(path)
# # Save
cluster_results.to_csv(path + "{}allclusters_summary_notwetlands.csv".format(region), index=False)

# Save
each_cells_value.to_csv(path + "{}individual_cell_values_notwetlands.csv".format(region), index=False)

In [32]:
each_cells_value

,short_id,Water_class,urban_class,Depth,Velocity,Hazard
299545,FEH,15.0,16.0,0.107,0.128,2.0
299546,FEH,15.0,16.0,0.106,0.129,2.0
305745,FEH,15.0,16.0,0.257,0.125,3.0
305746,FEH,15.0,16.0,0.215,0.122,2.0
305747,FEH,15.0,16.0,0.212,0.121,2.0
...,...,...,...,...,...,...
36472881,6h_c15,15.0,16.0,0.245,0.016,NaN
36479082,6h_c15,15.0,16.0,0.159,0.015,NaN
36485283,6h_c15,15.0,16.0,0.105,0.011,NaN
36485284,6h_c15,15.0,16.0,0.186,0.013,NaN
